In [28]:
import pandas as pd
import numpy as np
from typing import List, Dict, Tuple, Optional
import time
from pathlib import Path

print("Imports complete")

Imports complete


In [29]:
EMBEDDINGS_PATH = "/content/embeddings.parquet"  # or embeddings_series.parquet
RAW_DATA_PATH = "/content/raw_data.csv"
CLEANED_DATA_PATH = "/content/cleaned_data_names.parquet"

# Search parameters
DEFAULT_K = 10
DEFAULT_CANDIDATES = 100

print("Configuration set")

Configuration set



# DATA LOADING


In [30]:
def load_data():
    """Load all necessary data files"""
    print("Loading data files...")

    # Load embeddings
    emb_df = pd.read_parquet(EMBEDDINGS_PATH)
    embeddings = np.vstack(emb_df['embedding'].values)
    print(f"✓ Embeddings loaded: {embeddings.shape}")

    # Load metadata
    metadata = pd.read_parquet(CLEANED_DATA_PATH)
    print(f"✓ Metadata loaded: {len(metadata)} rows")

    # Load raw data for display
    raw_data = pd.read_csv(RAW_DATA_PATH)
    print("Check 1")
    raw_data["anime_id"] = pd.to_numeric(raw_data["anime_id"], errors="coerce")
    print(f"✓ Raw data loaded: {len(raw_data)} rows")

    # Verify alignment
    assert len(embeddings) == len(emb_df) == len(metadata), "Data size mismatch!"

    return embeddings, emb_df, metadata, raw_data

embeddings, emb_df, metadata, raw_data = load_data()

Loading data files...
✓ Embeddings loaded: (8857, 768)
✓ Metadata loaded: 8857 rows
Check 1
✓ Raw data loaded: 15000 rows


# KNN SEARCH FUNCTIONS

In [31]:
def cosine_similarity_batch(query_vec: np.ndarray,
                           embeddings: np.ndarray) -> np.ndarray:
    """
    Compute cosine similarity between query and all embeddings.
    Assumes both query_vec and embeddings are already L2-normalized.
    """
    return np.dot(embeddings, query_vec)


def knn_search(query_vec: np.ndarray,
               embeddings: np.ndarray,
               k: int = 10,
               exclude_idx: Optional[int] = None) -> Tuple[np.ndarray, np.ndarray]:
    """
    Find k nearest neighbors using cosine similarity.

    Returns:
        indices: array of k indices
        similarities: array of k similarity scores
    """
    sims = cosine_similarity_batch(query_vec, embeddings)

    # Get top-k
    K = min(k + (1 if exclude_idx is not None else 0), len(embeddings))
    top_indices = np.argpartition(-sims, K-1)[:K]
    top_indices = top_indices[np.argsort(-sims[top_indices])]

    # Exclude query index if specified
    if exclude_idx is not None:
        top_indices = top_indices[top_indices != exclude_idx]

    return top_indices[:k], sims[top_indices[:k]]


def approximate_knn_search(query_vec: np.ndarray,
                          embeddings: np.ndarray,
                          k: int = 10,
                          probe_size: int = 1000) -> Tuple[np.ndarray, np.ndarray]:
    """
    Faster approximate KNN using random sampling for very large datasets.
    Useful when exact search is too slow.
    """
    n = len(embeddings)
    if n <= probe_size or probe_size >= n:
        return knn_search(query_vec, embeddings, k)

    # Random sample
    sample_idx = np.random.choice(n, size=probe_size, replace=False)
    sample_emb = embeddings[sample_idx]

    # Search in sample
    local_top, local_sims = knn_search(query_vec, sample_emb, k)

    # Map back to original indices
    global_indices = sample_idx[local_top]
    return global_indices, local_sims

# QUERY PROCESSING

In [32]:
def find_anime_by_name(query_name: str,
                       metadata: pd.DataFrame,
                       search_columns: List[str] = None) -> Optional[int]:
    """
    Find anime index by searching in name columns.
    Returns the first match found.
    """
    if search_columns is None:
        search_columns = ["name", "english_name", "japanese_names", "embed_text"]

    for col in search_columns:
        if col not in metadata.columns:
            continue

        mask = metadata[col].astype(str).str.contains(
            query_name, case=False, na=False, regex=False
        )
        hits = metadata[mask].index.tolist()

        if hits:
            if len(hits) > 1:
                print(f"Found {len(hits)} matches in '{col}', using first match")
            return hits[0]

    return None


def create_query_embedding(query_text: str,
                          model) -> np.ndarray:
    """
    Create embedding for a text query using the same model.
    Requires: vertexai TextEmbeddingModel initialized
    """
    from vertexai.language_models import TextEmbeddingModel

    resp = model.get_embeddings([query_text])
    emb = np.asarray(resp[0].values, dtype=np.float32)

    # L2 normalize
    norm = np.linalg.norm(emb)
    if norm > 0:
        emb = emb / norm

    return emb

# RESULT FILTERING

In [33]:
def filter_same_series(candidate_indices: np.ndarray,
                      query_idx: int,
                      metadata: pd.DataFrame,
                      use_aggressive: bool = True) -> np.ndarray:
    """
    Remove sequels/related entries from the same series.
    Uses multiple strategies for robust filtering.

    Args:
        candidate_indices: Array of candidate indices to filter
        query_idx: Index of the query anime
        metadata: DataFrame with anime metadata
        use_aggressive: If True, uses more aggressive title matching
    """
    import re

    query_row = metadata.iloc[query_idx]
    query_name = str(query_row.get('name', '')).strip()

    if not query_name:
        return candidate_indices

    # Try series_key first
    if 'series_key' in metadata.columns:
        query_key = str(query_row.get('series_key', '')).strip().lower()

        def same_series(idx):
            if idx == query_idx:
                return True
            target_key = str(metadata.iloc[idx].get('series_key', '')).strip().lower()
            return query_key and target_key and (query_key == target_key)
    else:
        # Fallback to title matching
        query_title = str(query_row.get('name', '')).lower()
        query_root = ' '.join(query_title.split()[:2])

        def same_series(idx):
            if idx == query_idx:
                return True
            target_title = str(metadata.iloc[idx].get('name', '')).lower()
            return query_root and target_title.startswith(query_root)

    filtered = [idx for idx in candidate_indices if not same_series(idx)]
    return np.array(filtered)


def mmr_rerank(candidate_embeddings: np.ndarray,
               query_vec: np.ndarray,
               lambda_mult: float = 0.7,
               top_n: int = 10) -> List[int]:
    """
    Maximal Marginal Relevance re-ranking for diversity.

    Args:
        candidate_embeddings: (N, dim) embeddings of candidates
        query_vec: (dim,) query embedding
        lambda_mult: relevance vs diversity tradeoff (higher = more relevant)
        top_n: number of results to return
    """
    selected = []
    remaining = list(range(len(candidate_embeddings)))

    # Precompute query similarities
    query_sims = np.dot(candidate_embeddings, query_vec)

    for _ in range(min(top_n, len(candidate_embeddings))):
        if not remaining:
            break

        mmr_scores = []
        for idx in remaining:
            rel = query_sims[idx]  # Relevance to query

            if selected:
                # Diversity: max similarity to already selected
                selected_emb = candidate_embeddings[selected]
                div = np.max(np.dot(selected_emb, candidate_embeddings[idx]))
            else:
                div = 0

            mmr = lambda_mult * rel - (1 - lambda_mult) * div
            mmr_scores.append((mmr, idx))

        # Select best MMR score
        best_idx = max(mmr_scores, key=lambda x: x[0])[1]
        selected.append(best_idx)
        remaining.remove(best_idx)

    return selected


def diversify_by_genre(candidate_indices: List[int],
                      metadata: pd.DataFrame,
                      max_per_genre: int = 2) -> List[int]:
    """
    Ensure genre diversity by limiting entries per genre.
    """
    genre_counts = {}
    diverse = []

    for idx in candidate_indices:
        genres = metadata.iloc[idx].get('genre_theme_set', [])
        if isinstance(genres, str):
            genres = [genres]
        elif not isinstance(genres, (list, tuple)):
            genres = []

        main_genre = genres[0] if genres else "Unknown"

        if genre_counts.get(main_genre, 0) < max_per_genre:
            diverse.append(idx)
            genre_counts[main_genre] = genre_counts.get(main_genre, 0) + 1

    return diverse

# RECOMMENDATION MAIN

In [34]:
def recommend_anime(query_name: str,
                   embeddings: np.ndarray,
                   metadata: pd.DataFrame,
                   raw_data: pd.DataFrame,
                   k: int = 10,
                   candidates: int = 100,
                   use_mmr: bool = True,
                   lambda_mult: float = 0.7,
                   filter_series: bool = True,
                   diversify_genres: bool = True,
                   verbose: bool = True) -> List[Dict]:
    """
    Complete recommendation pipeline.

    Args:
        query_name: Anime title to search for
        embeddings: Pre-computed embeddings array
        metadata: DataFrame with anime metadata
        raw_data: DataFrame with display information
        k: Number of recommendations to return
        candidates: Number of initial candidates to consider
        use_mmr: Apply MMR re-ranking for diversity
        lambda_mult: MMR relevance weight (0-1)
        filter_series: Remove same-series entries
        diversify_genres: Ensure genre diversity
        verbose: Print detailed output

    Returns:
        List of recommendation dictionaries
    """
    start_time = time.time()

    # 1. Find query anime
    query_idx = find_anime_by_name(query_name, metadata)
    if query_idx is None:
        print(f"❌ '{query_name}' not found in database")
        return []

    query_row = metadata.iloc[query_idx]
    query_anime_id = query_row.get('anime_id')

    if verbose:
        print(f"📌 Query: {query_row.get('name', 'Unknown')} (ID: {query_anime_id}, Index: {query_idx})")
        print(f"{'='*80}")

    # 2. Get initial candidates
    query_vec = embeddings[query_idx]
    cand_indices, cand_sims = knn_search(
        query_vec, embeddings,
        k=candidates,
        exclude_idx=query_idx
    )

    # 3. Filter same series
    if filter_series:
        cand_indices = filter_same_series(cand_indices, query_idx, metadata)
        if len(cand_indices) == 0:
            print("⚠️  All candidates filtered out, using top results without filtering")
            cand_indices, _ = knn_search(query_vec, embeddings, k=k, exclude_idx=query_idx)

    # 4. MMR re-ranking
    if use_mmr and len(cand_indices) > k:
        cand_emb = embeddings[cand_indices]
        mmr_order = mmr_rerank(cand_emb, query_vec, lambda_mult, top_n=k*3)
        cand_indices = cand_indices[mmr_order]

    # 5. Genre diversity
    if diversify_genres:
        cand_indices = diversify_by_genre(cand_indices[:k*2], metadata, max_per_genre=3)

    # 6. Final selection
    final_indices = cand_indices[:k]

    # 7. Prepare results with lookups
    results = []
    for rank, idx in enumerate(final_indices, 1):
        anime_id = metadata.iloc[idx].get('anime_id')
        sim_score = cosine_similarity_batch(query_vec, embeddings[idx:idx+1])[0]

        # Lookup in raw_data
        info = raw_data[raw_data['anime_id'] == anime_id]
        if not info.empty:
            name = info.iloc[0].get('name', 'Unknown')
            score = info.iloc[0].get('score', 'N/A')
            episodes = info.iloc[0].get('episodes', 'N/A')
            genres = info.iloc[0].get('genres', 'N/A')
        else:
            name = metadata.iloc[idx].get('name', 'Unknown')
            score = metadata.iloc[idx].get('score', 'N/A')
            episodes = metadata.iloc[idx].get('episodes', 'N/A')
            genres = 'N/A'

        result = {
            'rank': rank,
            'anime_id': anime_id,
            'name': name,
            'similarity': float(sim_score),
            'score': score,
            'episodes': episodes,
            'genres': genres
        }
        results.append(result)

        if verbose:
            print(f"{rank:2d}. [{anime_id:6d}] {name[:50]:50s} | sim={sim_score:.4f} | score={score} | eps={episodes}")

    elapsed = time.time() - start_time
    if verbose:
        print(f"{'='*80}")
        print(f"⏱️  Search completed in {elapsed:.3f}s")

    return results

# TESTING

In [38]:
def test_series_filtering(query_name: str,
                         embeddings: np.ndarray,
                         metadata: pd.DataFrame,
                         top_n: int = 50):
    """
    Test function to verify same-series filtering is working.
    Shows what gets filtered and what remains.
    """
    print(f"\n{'='*80}")
    print(f"TESTING SERIES FILTER FOR: {query_name}")
    print(f"{'='*80}")

    # Find query
    query_idx = find_anime_by_name(query_name, metadata)
    if query_idx is None:
        print(f"❌ '{query_name}' not found")
        return

    query_row = metadata.iloc[query_idx]
    print(f"\n📌 Query Anime:")
    print(f"   Name: {query_row.get('name', 'Unknown')}")
    print(f"   ID: {query_row.get('anime_id')}")
    if 'series_key' in metadata.columns:
        print(f"   Series Key: {query_row.get('series_key', 'N/A')}")

    # Get candidates without filtering
    query_vec = embeddings[query_idx]
    cand_indices, cand_sims = knn_search(
        query_vec, embeddings,
        k=top_n,
        exclude_idx=query_idx
    )

    print(f"\n🔍 Top {top_n} candidates (before filtering):")
    print(f"{'='*80}")

    # Apply filtering
    filtered_indices = filter_same_series(cand_indices, query_idx, metadata, use_aggressive=True)
    filtered_set = set(filtered_indices)

    # Show results
    for i, idx in enumerate(cand_indices[:20], 1):  # Show top 20
        row = metadata.iloc[idx]
        name = row.get('name', 'Unknown')
        anime_id = row.get('anime_id', 'N/A')
        sim = cand_sims[i-1]

        is_filtered = idx not in filtered_set
        status = "❌ FILTERED" if is_filtered else "✅ KEPT"

        print(f"{i:2d}. [{anime_id:6}] {name[:60]:60s} | sim={sim:.4f} | {status}")

    print(f"\n{'='*80}")
    print(f"📊 Summary:")
    print(f"   Original candidates: {len(cand_indices)}")
    print(f"   Filtered out: {len(cand_indices) - len(filtered_indices)}")
    print(f"   Remaining: {len(filtered_indices)}")
    print(f"{'='*80}\n")
print("\n" + "="*80)
print("KNN SEARCH SYSTEM READY")
print("="*80)
test_series_filtering("One Piece", embeddings, metadata, top_n=50)
# Example 1: Basic recommendation
print("\n🔍 Example 1: Basic Recommendations")
results = recommend_anime("One Piece", embeddings, metadata, raw_data, k=10, filter_series = True)

# Example 2: More diverse results
print("\n🔍 Example 2: Diverse Recommendations (lower lambda)")
results = recommend_anime("One Piece", embeddings, metadata, raw_data,
                         k=10, lambda_mult=0.5)

# Example 3: Pure similarity (no MMR)
print("\n🔍 Example 3: Pure Similarity (no MMR)")
results = recommend_anime("One Piece", embeddings, metadata, raw_data,
                         k=10, use_mmr=False)


KNN SEARCH SYSTEM READY

TESTING SERIES FILTER FOR: One Piece
Found 22 matches in 'name', using first match

📌 Query Anime:
   Name: one piece movie 14 stampede
   ID: 38234

🔍 Top 50 candidates (before filtering):
 1. [ 60108] one piece gyojin tou hen                                     | sim=0.8622 | ❌ FILTERED
 2. [   459] one piece movie 01                                           | sim=0.8309 | ❌ FILTERED
 3. [ 31490] one piece film gold                                          | sim=0.8292 | ❌ FILTERED
 4. [  2107] one piece movie 08 episode of alabasta sabaku no oujo to kai | sim=0.8191 | ❌ FILTERED
 5. [  4155] one piece film strong world                                  | sim=0.8104 | ❌ FILTERED
 6. [   464] one piece movie 06 omatsuri danshaku to himitsu no shima     | sim=0.8073 | ❌ FILTERED
 7. [ 12859] one piece film z                                             | sim=0.8020 | ❌ FILTERED
 8. [   460] one piece movie 02 nejimaki jima no daibouken                | sim=0.79

# BATCH QUERY

In [37]:
def batch_recommend(anime_names: List[str], **kwargs) -> Dict[str, List[Dict]]:
    """
    Get recommendations for multiple anime at once.
    """
    results = {}
    for name in anime_names:
        print(f"\n{'='*80}")
        results[name] = recommend_anime(name, embeddings, metadata, raw_data,
                                       verbose=True, **kwargs)
    return results

#Example batch usage:
batch_results = batch_recommend(["Naruto", "Death Note", "Steins;Gate"], k=5)


Found 17 matches in 'name', using first match
📌 Query: naruto shippuuden (ID: 1735, Index: 295)
 1. [    20] Naruto                                             | sim=0.8825 | score=8.01 | eps=220.0
 2. [ 34566] Boruto: Naruto Next Generations                    | sim=0.8712 | score=5.99 | eps=293.0
 3. [ 10686] Naruto: Honoo no Chuunin Shiken! Naruto vs. Konoha | sim=0.8217 | score=7.17 | eps=1.0
⏱️  Search completed in 0.128s

📌 Query: death note (ID: 1535, Index: 90)
 1. [  5205] Kara no Kyoukai Movie 7: Satsujin Kousatsu (Go)    | sim=0.7384 | score=8.37 | eps=1.0
 2. [   269] Bleach                                             | sim=0.7237 | score=7.98 | eps=366.0
 3. [ 44961] Platinum End                                       | sim=0.7379 | score=6.02 | eps=24.0
⏱️  Search completed in 0.138s

❌ 'Steins;Gate' not found in database
